In [2]:
# imports
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Polygon
from shapely import wkt
import ast
import pyproj
from pyproj import Geod
import copy

In [3]:
# read datasets
poi_df = pd.read_csv('ca-geometry.csv')
patterns_df = pd.read_csv('2021-04-19-weekly-patterns.csv')
pop_df = pd.read_csv('ca-pop-da.csv')
home_df = pd.read_csv('2021-04-19-home-panel-summary.csv')
core_df = pd.read_csv('core_poi.csv')

In [4]:
# lambda functions
def home_cbg_filler(x):
    if x['home_cbgs']:
        return x['home_cbgs']
    else:
        return [x['poi_cbg']]

def cbg_val_filler(x):
    if x['cbgs_vals']:
        return x['cbgs_vals']
    else:
        return [1]

def origin_cbg_filler(x):
    return x['home_cbgs'][x['counter_col']]

def visitor_home_cbg_filler(x):
    return x['cbgs_vals'][x['counter_col']]

def normalized_visits(x):
    return [float(v)/sum(x['visits_by_each_hour']) for v in x['visits_by_each_hour']]

def true_hourly_visits(x):
    return [x['ext_visits_upper']*float(v) for v in x['normalized_hourly_visits']]

def extract_lon_lats_from_polygon_wkt(polygon_wkt):
    polygon_latlon = shapely.wkt.loads(polygon_wkt)
    polygon_points = list(polygon_latlon.exterior.coords)
    lon, lat = zip(*polygon_points)
    return(lon,lat)

def get_geodesic_area(polygon_wkt, ellps_model='IAU76'):
    # Default uses model of Earth IAU 1976 https://en.wikipedia.org/wiki/IAU_(1976)_System_of_Astronomical_Constants 
    geod = Geod(ellps=ellps_model)
    lon, lat = extract_lon_lats_from_polygon_wkt(polygon_wkt)
    poly_area, poly_perimeter = geod.polygon_area_perimeter(lon, lat) # in square meters
#     square_feet_meter_conv = 10.7639 # square feet in 1 square meter
#     poly_area = poly_area * square_feet_meter_conv
    return(abs(poly_area))

def tuple_key(x):
    poi_cbg = str(x['poi_cbg'])
    return (x['location_name'], poi_cbg)

def comp_poi_risk(x):
    return [v*x['median_dwell']/np.sqrt(x['geodesic_area']) for v in x['true_hourly_visits']]

In [5]:
# get the population per dissemination area (cbg)
pop_df.rename(columns={'Population, 2016': 'population', 'Geographic code':'dissemination_area'}, inplace=True)
pop_df.drop(pop_df.columns.difference(['dissemination_area','population']), 1, inplace=True)
pop_df = pop_df[:-6] # remove some garbage
pop_df['dissemination_area'] = pop_df.dissemination_area.apply(int)

# modify NaNs and 0s
prev_count = 0
curr_count = 0
missing_code_list = []
for i in range(pop_df.shape[0]):
    if not np.isnan(pop_df.iloc[i,1]) and pop_df.iloc[i,1] > 0:
        curr_count = pop_df.iloc[i,1]
        new_pop = np.mean([prev_count, curr_count])
        for code in missing_code_list:
            pop_df.loc[(pop_df.dissemination_area == code), 'population'] = new_pop
        prev_count = curr_count
        missing_code_list = []
    else:
        missing_code_list.append(pop_df.iloc[i,0])

pop_df['population'] = pop_df.population.apply(int)
pop_df['dissemination_area'] = pop_df.dissemination_area.apply(str)
pop_df.head()

,dissemination_area,population
0,10010165,506
1,10010166,327
2,10010167,431
3,10010168,524
4,10010169,311


In [6]:
patterns_temp = patterns_df.copy()

In [7]:
patterns_list_columns = ['location_name','raw_visit_counts', 'raw_visitor_counts', 'visits_by_day',
                         'visits_by_each_hour', 'poi_cbg', 'visitor_home_cbgs', 'median_dwell']
patterns_df['poi_cbg'] = patterns_df.poi_cbg.apply(lambda x: int(x[3:]))
patterns_df.drop(patterns_df.columns.difference(patterns_list_columns), 1, inplace=True)

expdf = patterns_df.copy()
expdf.drop(['median_dwell'], axis=1, inplace=True)
expdf['origin_cbg'] = None

cols = expdf.columns.tolist()
cols = cols[0:1] + cols[-1:] + cols[1:-1]
expdf = expdf[cols]

expdf['visitor_home_cbgs'] = expdf.visitor_home_cbgs.apply(eval)
expdf['count_col'] = expdf.visitor_home_cbgs.apply(len)

expdf['count_col'] = expdf.count_col.apply(lambda x: max(x,1))
expdf['home_cbgs'] = expdf.visitor_home_cbgs.apply(list)
expdf['home_cbgs'] = expdf.home_cbgs.apply(lambda x: [c[3:] for c in x])
expdf['cbgs_vals'] = expdf.visitor_home_cbgs.apply(lambda x: list(x.values()))

expdf['home_cbgs'] = expdf.apply(home_cbg_filler, axis=1)
expdf['cbgs_vals'] = expdf.apply(cbg_val_filler, axis=1)

# counter_col
expdf = expdf.loc[expdf.index.repeat(expdf['count_col'])]
expdf.insert(0, 'counter_col', expdf.groupby(level=0).cumcount())
expdf = expdf.reset_index(drop=True)

expdf['origin_cbg'] = expdf.apply(origin_cbg_filler, axis=1)
expdf['visitor_home_cbgs'] = expdf.apply(visitor_home_cbg_filler, axis=1)

expdf['est_visits_per_visitor'] = expdf['raw_visit_counts']/expdf['raw_visitor_counts']
expdf['visitor_home_lower'] = expdf.visitor_home_cbgs.apply(lambda x: 2 if x == 4 else x)
expdf['visitor_home_upper'] = expdf.visitor_home_cbgs.apply(lambda x: 4 if x == 4 else x)

# set checkpoint
temp_expdf = expdf.copy()
expdf.head()

,counter_col,location_name,origin_cbg,raw_visit_counts,raw_visitor_counts,visits_by_day,visits_by_each_hour,poi_cbg,visitor_home_cbgs,count_col,home_cbgs,cbgs_vals,est_visits_per_visitor,visitor_home_lower,visitor_home_upper
0,0,Shell Oil,35202879,10,9,"[0,3,0,2,3,1,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",35202879,1,1,[35202879],[1],1.111111,1,1
1,0,Colborne Lodge,35204627,6,6,"[0,0,2,0,2,0,2]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",35201312,4,1,[35204627],[4],1.000000,2,4
2,0,Ormstown Veterinary Clinic Regd,24690057,1,1,"[0,0,0,1,0,0,0]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",24690057,1,1,[24690057],[1],1.000000,1,1
3,0,Stokes Stores,48112267,11,11,"[3,1,2,0,1,2,2]","[0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0...",48111463,4,3,"[48112267, 48111445, 48110400]","[4, 4, 4]",1.000000,2,4
4,1,Stokes Stores,48111445,11,11,"[3,1,2,0,1,2,2]","[0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0...",48111463,4,3,"[48112267, 48111445, 48110400]","[4, 4, 4]",1.000000,2,4


In [8]:
# extract cbg and number_devices_residing (SG sample number) from home_df
home_columns = ['census_block_group', 'number_devices_residing']
home_df.drop(home_df.columns.difference(home_columns), 1, inplace=True)
home_df['census_block_group'] = home_df.census_block_group.apply(lambda x: str(x[3:]))
home_df.head()

,census_block_group,number_devices_residing
0,48060695,10
1,48060240,5
2,48110207,4
3,48110853,5
4,48150143,3


In [9]:
expdf = temp_expdf.copy()

In [10]:
# join expdf and home_df
expdf = expdf.join(home_df.set_index('census_block_group'), on='origin_cbg')
expdf['number_devices_residing'] = expdf['number_devices_residing'].fillna(1)
expdf['number_devices_residing'] = expdf.number_devices_residing.apply(int)

# join expdf and pop_df
expdf['origin_cbg'] = expdf.origin_cbg.apply(str)
expdf = expdf.join(pop_df.set_index('dissemination_area'), on='origin_cbg')
expdf = expdf[~expdf['population'].isnull()]
expdf['population'] = expdf.population.apply(int)

# filling extrapolated visitor and visits
expdf = expdf.rename(columns={'number_devices_residing':'safegraph_sample_size'})
expdf['scale_factor'] = expdf['population']/expdf['safegraph_sample_size']
expdf['ext_visitor_upper'] = expdf['scale_factor']*expdf['visitor_home_upper']
expdf['ext_visits_upper'] = expdf['ext_visitor_upper']*expdf['est_visits_per_visitor']
expdf.head()

,counter_col,location_name,origin_cbg,raw_visit_counts,raw_visitor_counts,visits_by_day,visits_by_each_hour,poi_cbg,visitor_home_cbgs,count_col,home_cbgs,cbgs_vals,est_visits_per_visitor,visitor_home_lower,visitor_home_upper,safegraph_sample_size,population,scale_factor,ext_visitor_upper,ext_visits_upper
0,0,Shell Oil,35202879,10,9,"[0,3,0,2,3,1,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",35202879,1,1,[35202879],[1],1.111111,1,1,1,1343,1343.000000,1343.000000,1492.222222
1,0,Colborne Lodge,35204627,6,6,"[0,0,2,0,2,0,2]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",35201312,4,1,[35204627],[4],1.000000,2,4,6,437,72.833333,291.333333,291.333333
2,0,Ormstown Veterinary Clinic Regd,24690057,1,1,"[0,0,0,1,0,0,0]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",24690057,1,1,[24690057],[1],1.000000,1,1,1,494,494.000000,494.000000,494.000000
3,0,Stokes Stores,48112267,11,11,"[3,1,2,0,1,2,2]","[0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0...",48111463,4,3,"[48112267, 48111445, 48110400]","[4, 4, 4]",1.000000,2,4,24,2036,84.833333,339.333333,339.333333
4,1,Stokes Stores,48111445,11,11,"[3,1,2,0,1,2,2]","[0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0...",48111463,4,3,"[48112267, 48111445, 48110400]","[4, 4, 4]",1.000000,2,4,6,287,47.833333,191.333333,191.333333


In [11]:
# drop unnecessary columns
expdf = expdf.drop(columns=['counter_col', 'origin_cbg', 'home_cbgs', 'cbgs_vals', 'visitor_home_lower',
                           'visitor_home_upper', 'safegraph_sample_size', 'population', 'scale_factor'])

# adding the cum_sum column -- used to perform groupby for consecutive same rows
expdf['shift'] = expdf.poi_cbg.shift(1)
expdf['b_val'] = expdf['poi_cbg'] != expdf['shift']
expdf['cum_sum'] = expdf.b_val.cumsum()
expdf['cum_sum'] = expdf['cum_sum']-1

expdf.head()

,location_name,raw_visit_counts,raw_visitor_counts,visits_by_day,visits_by_each_hour,poi_cbg,visitor_home_cbgs,count_col,est_visits_per_visitor,ext_visitor_upper,ext_visits_upper,shift,b_val,cum_sum
0,Shell Oil,10,9,"[0,3,0,2,3,1,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",35202879,1,1,1.111111,1343.000000,1492.222222,NaN,True,0
1,Colborne Lodge,6,6,"[0,0,2,0,2,0,2]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",35201312,4,1,1.000000,291.333333,291.333333,35202879.0,True,1
2,Ormstown Veterinary Clinic Regd,1,1,"[0,0,0,1,0,0,0]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",24690057,1,1,1.000000,494.000000,494.000000,35201312.0,True,2
3,Stokes Stores,11,11,"[3,1,2,0,1,2,2]","[0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0...",48111463,4,3,1.000000,339.333333,339.333333,24690057.0,True,3
4,Stokes Stores,11,11,"[3,1,2,0,1,2,2]","[0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0...",48111463,4,3,1.000000,191.333333,191.333333,48111463.0,False,3


In [12]:
# take only specific columns bec we want to aggregate (sum) rows having same cum_sum
# then do group by and reset the indices
expdf_copy = expdf.drop(expdf.columns.difference(['location_name','poi_cbg', 'cum_sum', 'ext_visitor_upper', 'ext_visits_upper']), 1)
expdf_copy = expdf_copy.groupby(['cum_sum', 'poi_cbg', 'location_name']).sum().reset_index()

expdf_copy.head()

,cum_sum,poi_cbg,location_name,ext_visitor_upper,ext_visits_upper
0,0,35202879,Shell Oil,1343.000000,1492.222222
1,1,35201312,Colborne Lodge,291.333333,291.333333
2,2,24690057,Ormstown Veterinary Clinic Regd,494.000000,494.000000
3,3,48111463,Stokes Stores,904.952381,904.952381
4,4,48110221,Tim Hortons,1147.698630,1377.238356


In [13]:
# get only specific columns
expdf_copy5 = pd.DataFrame(columns=['ext_visitor_upper','ext_visits_upper'])
expdf_copy5[['ext_visitor_upper','ext_visits_upper']] = expdf_copy[['ext_visitor_upper','ext_visits_upper']]

expdf_copy3 = expdf.drop(expdf.columns.difference(['location_name','poi_cbg', 'cum_sum', 'visits_by_each_hour']), 1)
expdf_copy4 = expdf_copy3.drop_duplicates()
expdf_copy4 = expdf_copy4.reset_index().drop(columns=['index'])
extrapolation_df = pd.concat([expdf_copy4, expdf_copy5], axis=1)

# checkpoint
tempextrap = extrapolation_df.copy()

extrapolation_df.head()

,location_name,visits_by_each_hour,poi_cbg,cum_sum,ext_visitor_upper,ext_visits_upper
0,Shell Oil,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",35202879,0,1343.000000,1492.222222
1,Colborne Lodge,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",35201312,1,291.333333,291.333333
2,Ormstown Veterinary Clinic Regd,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",24690057,2,494.000000,494.000000
3,Stokes Stores,"[0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0...",48111463,3,904.952381,904.952381
4,Tim Hortons,"[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0...",48110221,4,1147.698630,1377.238356


In [14]:
tempextrap = extrapolation_df.copy()

In [15]:
# need the extrapolation_df
extrapolation_df['visits_by_each_hour'] = extrapolation_df.visits_by_each_hour.apply(eval)
extrapolation_df['total_visits'] = extrapolation_df.visits_by_each_hour.apply(sum)
extrapolation_df['total_visits'] = extrapolation_df.total_visits.apply(float)
extrapolation_df['normalized_hourly_visits'] = extrapolation_df.apply(normalized_visits, axis=1)
extrapolation_df['true_hourly_visits'] = extrapolation_df.apply(true_hourly_visits, axis=1)

extrapolation_df.head() # final df for the extrapolation
# extrapolation_df.to_csv('extrapolation-2021-04-19.csv', index=False) # save if needed

In [16]:
patterns_df = patterns_temp.copy()
patterns_df['poi_cbg'] = patterns_df.poi_cbg.apply(str)
patterns_df['poi_cbg'] = patterns_df.poi_cbg.apply(lambda x: x[3:])
patterns_df['tup_key'] = patterns_df.apply(tuple_key, axis=1)
patterns_df.head()

,safegraph_place_id,location_name,street_address,city,region,postal_code,iso_country_code,safegraph_brand_ids,brands,date_range_start,...,visitor_daytime_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_week_brand,device_type,carrier_name,tup_key
0,sg:00112a9a83e34a19a398e278ecafd18f,Shell Oil,1077 Yonge Street,Toronto,ON,M4W 2L5,CA,SG_BRAND_2591c98b70119fe624898b1e424b5e91,Shell Oil,2021-04-19T00:00:00-04:00,...,{},"{""CA"":7}",8127.0,10.0,"{""<5"":2,""5-10"":4,""11-20"":2,""21-60"":1,""61-120"":...","{""Sobeys"":50}","{""Esso"":41,""McDonald's"":27,""Tim Hortons"":24,""P...","{""android"":4,""ios"":7}",{},"(Shell Oil, 35202879)"
1,sg:00c005ad029644f59ea9ca1a050242d0,Colborne Lodge,11 Colborne Lodge Drive,Toronto,ON,M3C 0C1,CA,NaN,NaN,2021-04-19T00:00:00-04:00,...,{},"{""CA"":4}",NaN,11.5,"{""<5"":0,""5-10"":3,""11-20"":2,""21-60"":1,""61-120"":...",{},"{""Starbucks"":31,""Nofrills"":16,""Shoppers Drug M...","{""android"":4,""ios"":0}",{},"(Colborne Lodge, 35201312)"
2,sg:00efd204df6f4602af60161a34451fb7,Ormstown Veterinary Clinic Regd,15 Rue Gale,Ormstown,QC,J0S 1K0,CA,NaN,NaN,2021-04-19T00:00:00-04:00,...,{},NaN,NaN,27.0,"{""<5"":0,""5-10"":0,""11-20"":0,""21-60"":1,""61-120"":...",{},{},"{""android"":0,""ios"":0}",{},"(Ormstown Veterinary Clinic Regd, 24690057)"
3,sg:0132a7f1c8733799a65789ca2637f363,Stokes Stores,137 Th Avenue And 66th Street,Edmonton,AB,T5C 3C8,CA,SG_BRAND_9ed1019bc28d739f,Stokes Stores,2021-04-19T00:00:00-06:00,...,"{""CA:48110400"":4,""CA:48111592"":4,""CA:48112270"":4}","{""CA"":9}",3592.0,24.0,"{""<5"":0,""5-10"":1,""11-20"":3,""21-60"":6,""61-120"":...","{""Impark"":100,""World Health"":50,""McDonald's"":47}","{""McDonald's"":40,""Canadian Tire"":34,""Costco"":3...","{""android"":0,""ios"":4}",{},"(Stokes Stores, 48111463)"
4,sg:0141c576aaf94f82ab18586dbca7dfc9,Tim Hortons,11763 106 Street,Edmonton,AB,T5G 2R1,CA,SG_BRAND_c4c485492c7ab7cb854a7c2b9772cfa1,Tim Hortons,2021-04-19T00:00:00-06:00,...,"{""CA:48111970"":4,""CA:48112186"":4}","{""CA"":5}",15384.0,105.5,"{""<5"":0,""5-10"":2,""11-20"":0,""21-60"":1,""61-120"":...",{},"{""Dollarama"":56,""Safeway Pharmacy"":40,""Costco""...","{""android"":5,""ios"":4}",{},"(Tim Hortons, 48110221)"


In [17]:
extrapolation_df['tup_key'] = extrapolation_df.apply(tuple_key, axis=1)

dwell_df = pd.DataFrame(columns=['tup_key', 'safegraph_place_id', 'median_dwell'])
dwell_df[['tup_key', 'safegraph_place_id', 'median_dwell']] = patterns_df[['tup_key', 'safegraph_place_id', 'median_dwell']]

extrapolation_dwell_df = extrapolation_df.join(dwell_df.set_index('tup_key'), on='tup_key')
extrapolation_dwell_df.head()

,location_name,visits_by_each_hour,poi_cbg,cum_sum,ext_visitor_upper,ext_visits_upper,total_visits,normalized_hourly_visits,true_hourly_visits,tup_key,safegraph_place_id,median_dwell
0,Shell Oil,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",35202879,0,1343.000000,1492.222222,10.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(Shell Oil, 35202879)",sg:00112a9a83e34a19a398e278ecafd18f,10.0
1,Colborne Lodge,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",35201312,1,291.333333,291.333333,6.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(Colborne Lodge, 35201312)",sg:00c005ad029644f59ea9ca1a050242d0,11.5
2,Ormstown Veterinary Clinic Regd,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",24690057,2,494.000000,494.000000,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(Ormstown Veterinary Clinic Regd, 24690057)",sg:00efd204df6f4602af60161a34451fb7,27.0
3,Stokes Stores,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...",48111463,3,904.952381,904.952381,11.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(Stokes Stores, 48111463)",sg:0132a7f1c8733799a65789ca2637f363,24.0
4,Tim Hortons,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",48110221,4,1147.698630,1377.238356,6.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(Tim Hortons, 48110221)",sg:0141c576aaf94f82ab18586dbca7dfc9,105.5


In [18]:
# preprocess
poi_df = poi_df[poi_df['iso_country_code'] == 'CA']
poi_df['bool_poly'] = poi_df.polygon_wkt.apply(lambda x: 'MULTIPOLYGON' in x)
poi_df = poi_df[~poi_df['bool_poly']]
poi_df = poi_df.drop(columns=['bool_poly', 'location_name'])

# get area in sq metres
poi_df['geodesic_area'] = poi_df['polygon_wkt'].apply(get_geodesic_area, ellps_model='IAU76')
poi_df.head()

,placekey,safegraph_place_id,parent_placekey,parent_safegraph_place_id,brands,latitude,longitude,street_address,city,region,postal_code,iso_country_code,polygon_wkt,polygon_class,includes_parking_lot,is_synthetic,building_height,enclosed,geodesic_area
0,zzw-222@65x-9m6-qfz,sg:006a0dff74f34238b25186af152aebb8,NaN,NaN,NaN,45.414467,-75.640502,767 Belfast Road,Ottawa,ON,K1G 0Z4,CA,"POLYGON ((-75.64022410254003 45.4144569, -75.6...",SHARED_POLYGON,NaN,True,NaN,False,1236.041174
1,225-228@3x2-dbc-4d9,sg:022dd07a6d984332a3c1e1948cab32fe,NaN,NaN,NaN,49.888145,-119.491533,1433 Street Paul Street,Kelowna,BC,V1Y 2E4,CA,"POLYGON ((-119.491816 49.887287, -119.491841 4...",SHARED_POLYGON,False,False,NaN,False,3195.008509
2,22d-222@3wx-yd9-c5z,sg:03eef9a4da2f4fa1a062dc3e4228ae72,NaN,NaN,NaN,49.695353,-112.839831,313 5 Street South,Lethbridge,AB,T1J 2B4,CA,POLYGON ((-112.8400147295265 49.69531000107685...,OWNED_POLYGON,False,False,NaN,False,138.422287
3,zzy-222@5x6-rwf-r49,sg:0433d53e40c04dc49241b2e0db08e96d,NaN,NaN,NaN,49.060987,-122.485332,32 A Avenue,Aldergrove,BC,V4W 3G1,CA,"POLYGON ((-122.485247 49.061025, -122.485247 4...",OWNED_POLYGON,False,False,NaN,False,121.694143
4,222-222@65w-kzd-5xq,sg:045dab58ab2b4f1696496eebcb0950d7,NaN,NaN,NaN,45.006716,-74.730580,17349 Cornwall Island,Cornwall,ON,K6H 5R7,CA,POLYGON ((-74.73035987683988 45.00665832823929...,SHARED_POLYGON,False,False,NaN,False,3685.003622


In [19]:
# join extrapolation_dwell_df and poi_df
poi_risk_df = poi_df.join(extrapolation_dwell_df.set_index('safegraph_place_id'), on='safegraph_place_id')

poi_risk_df = poi_risk_df[poi_risk_df['median_dwell'].notna()]
poi_risk_df = poi_risk_df[poi_risk_df['true_hourly_visits'].notna()]
poi_risk_df = poi_risk_df[poi_risk_df['geodesic_area'].notna()]

poi_risk_df['poi_risk'] = poi_risk_df.apply(comp_poi_risk, axis=1)
poi_risk_df = poi_risk_df.reset_index().drop(columns=['index'])


poi_risk_df.head()

,placekey,safegraph_place_id,parent_placekey,parent_safegraph_place_id,brands,latitude,longitude,street_address,city,region,...,poi_cbg,cum_sum,ext_visitor_upper,ext_visits_upper,total_visits,normalized_hourly_visits,true_hourly_visits,tup_key,median_dwell,poi_risk
0,zzw-222@65x-9m6-qfz,sg:006a0dff74f34238b25186af152aebb8,NaN,NaN,NaN,45.414467,-75.640502,767 Belfast Road,Ottawa,ON,...,35060991.0,122558.0,602.000000,602.000000,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(Demenagement & Entreposage Boyd Ltee Membre D...,52.5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,22d-222@3wx-yd9-c5z,sg:03eef9a4da2f4fa1a062dc3e4228ae72,NaN,NaN,NaN,49.695353,-112.839831,313 5 Street South,Lethbridge,AB,...,48020204.0,58824.0,413.000000,413.000000,3.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(Dylan's Burger & Deli, 48020204)",9.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,222-222@66b-2dy-ht9,sg:04b12c3a6eba4a6c8b40a1c1cdba1243,NaN,NaN,NaN,45.567055,-73.686813,322 Saint Hubert Rue,Laval,QC,...,24650199.0,281406.0,532.363636,532.363636,2.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(Alex Plomberie & Chauffage Enrg, 24650199)",737.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,zzw-222@5sz-yq2-zxq,sg:0556e5faae5449429835ce62a6071bbd,NaN,NaN,NaN,50.489690,-104.631866,873 North Argyle Street,Regina,SK,...,47060450.0,262280.0,822.000000,1233.000000,15.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0666666666666...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 82.2, 0.0, 0.0,...","(Klein's Food Mart, 47060450)",17.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 86.920440858338..."
4,222-222@665-zky-p7q,sg:0605aff4eae14a4bb50b84f634ca7384,NaN,NaN,NaN,43.595813,-79.623489,1250 Mississauga Valley Boulevard Unit 164,Mississauga,ON,...,35211827.0,141636.0,4133.943087,6239.914094,80.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0125, 0.0125,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 77.998926170338...","(Side B Vibe, 35211827)",45.5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.4492910406906..."


In [20]:
poi_risk_dataframe = poi_risk_df[['placekey', 'safegraph_place_id', 'location_name', 'latitude', 'longitude', 'poi_risk']]
core_df = core_df[['placekey', 'top_category']]
poi_risk_dataframe = pd.merge(poi_risk_dataframe, core_df, on='placekey')

poi_risk_dataframe.head()

,placekey,safegraph_place_id,location_name,latitude,longitude,poi_risk,top_category
0,zzw-222@65x-9m6-qfz,sg:006a0dff74f34238b25186af152aebb8,Demenagement & Entreposage Boyd Ltee Membre De...,45.414467,-75.640502,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Specialized Freight Trucking
1,22d-222@3wx-yd9-c5z,sg:03eef9a4da2f4fa1a062dc3e4228ae72,Dylan's Burger & Deli,49.695353,-112.839831,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Restaurants and Other Eating Places
2,222-222@66b-2dy-ht9,sg:04b12c3a6eba4a6c8b40a1c1cdba1243,Alex Plomberie & Chauffage Enrg,45.567055,-73.686813,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Building Equipment Contractors
3,zzw-222@5sz-yq2-zxq,sg:0556e5faae5449429835ce62a6071bbd,Klein's Food Mart,50.489690,-104.631866,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 86.920440858338...",Grocery Stores
4,222-222@665-zky-p7q,sg:0605aff4eae14a4bb50b84f634ca7384,Side B Vibe,43.595813,-79.623489,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.4492910406906...","Promoters of Performing Arts, Sports, and Simi..."


In [21]:
for i in range(24*7):
    name = 'poiRisk_' + str(i)
    poi_risk_dataframe[name] = poi_risk_dataframe.poi_risk.apply(lambda x: x[i])

poi_risk_dataframe = poi_risk_dataframe.drop(columns=['poi_risk'])

poi_risk_dataframe.to_csv('ca_poi_risks_2021-04-19-one-week.csv', index=False) #save
poi_risk_dataframe.head()

,placekey,safegraph_place_id,location_name,latitude,longitude,top_category,poiRisk_0,poiRisk_1,poiRisk_2,poiRisk_3,...,poiRisk_158,poiRisk_159,poiRisk_160,poiRisk_161,poiRisk_162,poiRisk_163,poiRisk_164,poiRisk_165,poiRisk_166,poiRisk_167
0,zzw-222@65x-9m6-qfz,sg:006a0dff74f34238b25186af152aebb8,Demenagement & Entreposage Boyd Ltee Membre De...,45.414467,-75.640502,Specialized Freight Trucking,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,22d-222@3wx-yd9-c5z,sg:03eef9a4da2f4fa1a062dc3e4228ae72,Dylan's Burger & Deli,49.695353,-112.839831,Restaurants and Other Eating Places,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,222-222@66b-2dy-ht9,sg:04b12c3a6eba4a6c8b40a1c1cdba1243,Alex Plomberie & Chauffage Enrg,45.567055,-73.686813,Building Equipment Contractors,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,zzw-222@5sz-yq2-zxq,sg:0556e5faae5449429835ce62a6071bbd,Klein's Food Mart,50.489690,-104.631866,Grocery Stores,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,222-222@665-zky-p7q,sg:0605aff4eae14a4bb50b84f634ca7384,Side B Vibe,43.595813,-79.623489,"Promoters of Performing Arts, Sports, and Simi...",0.0,0.0,0.0,0.0,...,16.898582,8.449291,0.0,0.0,0.0,0.0,33.797164,0.0,0.0,8.449291


In [22]:
poi_risk_dataframe[poi_risk_dataframe['latitude'] == 43.794774]

,placekey,safegraph_place_id,location_name,latitude,longitude,top_category,poiRisk_0,poiRisk_1,poiRisk_2,poiRisk_3,...,poiRisk_158,poiRisk_159,poiRisk_160,poiRisk_161,poiRisk_162,poiRisk_163,poiRisk_164,poiRisk_165,poiRisk_166,poiRisk_167
243526,223-223@665-z7y-ch5,sg:3f7ea9262eed465fbfb8f85231fb05af,Nofrills,43.794774,-79.421871,Grocery Stores,0.0,0.0,0.0,0.0,...,0.0,9.342747,0.0,9.342747,9.342747,9.342747,9.342747,0.0,0.0,0.0


In [23]:
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 2)
extrapolation_dwell_df

,location_name,visits_by_each_hour,poi_cbg,cum_sum,ext_visitor_upper,ext_visits_upper,total_visits,normalized_hourly_visits,true_hourly_visits,tup_key,safegraph_place_id,median_dwell
0,Shell Oil,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",35202879,0,1343.0,1492.222222,10.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(Shell Oil, 35202879)",sg:00112a9a83e34a19a398e278ecafd18f,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
283921,New Penny Family Restaurant,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",35430748,283871,494.0,494.000000,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(New Penny Family Restaurant, 35430748)",sg:ffab4a3130f04beb868a1f18b9d92293,88.0


In [24]:
pd.set_option("display.max_columns", None)
# poi_risk_df.head()
poi_risk_df[['safegraph_place_id', 'geodesic_area']]

,safegraph_place_id,geodesic_area
0,sg:006a0dff74f34238b25186af152aebb8,1236.041174
...,...,...
288127,sg:ffb1474af9554251a3a1f9573f45ea51,135.538418


In [25]:
tmp = pd.merge(extrapolation_dwell_df, poi_risk_df[['safegraph_place_id', 'geodesic_area', 'latitude', 'longitude']], on='safegraph_place_id')
tmp = pd.merge(tmp, patterns_df[['safegraph_place_id', 'bucketed_dwell_times']], on='safegraph_place_id')

bound_1 = [43.822708, -80.044187]
bound_2 = [43.768655, -78.674769]

print(len(tmp))
tmp = tmp[tmp['latitude'] <= bound_1[0]][tmp['latitude'] >= bound_2[0]]
tmp = tmp[tmp['longitude'] >= bound_1[1]][tmp['longitude'] <= bound_2[1]]
tmp.reset_index(drop=True, inplace=True)
print(len(tmp))

tmp.to_csv("tmp_file_for_risk_processing.csv")

# This file would be used in POI_risk_calculator.ipynb for further processing of risk

305026
7970


<ipython-input-25-91c86a83f675>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp = tmp[tmp['latitude'] <= bound_1[0]][tmp['latitude'] >= bound_2[0]]
<ipython-input-25-91c86a83f675>:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp = tmp[tmp['longitude'] >= bound_1[1]][tmp['longitude'] <= bound_2[1]]
